In [1]:
# Tratamiento de datos
import numpy as np
import pandas as pd

# Preprocesado y modelado
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#Configuración warnings
import warnings
warnings.filterwarnings('once')

In [2]:
df_casual = pd.read_pickle('../../../datos_finales/archivo_casual_est_enc2.pkl')
df_casual.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,usuarios_casuales
0,1.0,1.0,1.0,2.67,1.79,4.25,-0.486274,0.855582,-0.208909,331
1,1.0,1.0,1.0,1.83,1.00,4.25,-0.425690,0.332722,0.687550,131


In [59]:
lista_cols = ['dias_laborables', 'tiempo']

In [60]:
# Decidimos quitar las columnas porque tenemos overfitting.
df_casual.drop(lista_cols, axis = 1, inplace = True)

In [61]:
X = df_casual.drop("usuarios_casuales", axis = 1)
y = df_casual["usuarios_casuales"]

In [62]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [63]:
param = {"max_depth": [4,6,8], 
        "max_features": [1,2,3],
        'subsample'     : [0.5, 1], # parametros que se usan de forma estandar
        'learning_rate' : [0.001, 0.01, 0.1]} # parametros que se usan de forma estandar

In [64]:
gs_gb = GridSearchCV(
            estimator=GradientBoostingRegressor(), 
            param_grid= param, 
            cv=10, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [65]:
gs_gb.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [4, 6, 8], 'max_features': [1, 2, 3],
                         'subsample': [0.5, 1]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [66]:
boosting = gs_gb.best_estimator_
boosting

GradientBoostingRegressor(max_depth=6, max_features=2, subsample=1)

In [67]:
y_pred_test_gb = boosting.predict(x_test)
y_pred_train_gb = boosting.predict(x_train)

In [68]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    """
    Esta función nos dice las métricas de nuestro modelo.
    Args:
        y_test (Serie): el test de nuestra variable respuesta.
        y_train (Serie): el train de nuestra variable respuesta.
        y_test_pred (Serie): la predicción para el test de nuestra variable respuesta.
        y_train_pred (Serie): la predicción para el train de nuestra variable respuesta.
        tipo_modelo (str): nombre que le damos al modelo.
    Returns:
        dataframe con todas las métricas de nuestro modelo.
    """
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [69]:
dt_results = metricas(y_test, y_train, y_pred_test_gb, y_pred_train_gb, "Gradient Boosting")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,193.874097,84400.351994,290.517387,0.824321,test,Gradient Boosting
1,40.290683,3217.416467,56.722275,0.993121,train,Gradient Boosting


Este modelo nos devuelve una métrica muy buena para R2 pero hay demasiado overfitting.

In [70]:

importancia_predictores = pd.DataFrame(
                            {'predictor': x_train.columns,
                             'importancia': boosting.feature_importances_}
                            )


importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
3,dia_semana,0.264058
4,temperatura,0.220893
0,estacion,0.132437
2,mes,0.126817
5,humedad,0.099442
6,velocidad_viento,0.098641
1,año,0.057712
